In [1]:
import re
import pandas as pd
import numpy as np

# from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
#  Sequential,
from tensorflow.keras.layers import Input, Dense, LSTM, Conv1D, Embedding, LSTM, Flatten
from tensorflow.keras.optimizers import Adam
#  GRU, Bidirectional
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# from transformers import BertTokenizerFast, TFBertModel
from sentence_transformers import SentenceTransformer
# from transformers import DistilBertTokenizer, TFDistilBertModel

# from scripts.word_embeddings import load_embedding_weights

# Defining utility functions

In [2]:
def clean(word):
    return re.sub(r'[\-\(\)\?\*\\\'\"\.%\^,:<>|;!]', '', word.lower())

# Preprocessing

In [2]:
path = '/home/petar/Fakultet/Semester 7/NLP/Datasets/fake_news/'

In [3]:
train = pd.read_csv(path + 'full_train_df.csv', index_col=0)
test = pd.read_csv(path + 'full_test_df.csv', index_col=0)

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29999 entries, 0 to 29998
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   user    29999 non-null  object
 1   tweet   29999 non-null  object
 2   label   29999 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 937.5+ KB


In [5]:
train.head()

,user,tweet,label
0,fav48xfxollnr5ho5pf5e2489ts2vhll,In light of the Democrats did not have been bo...,1
1,fav48xfxollnr5ho5pf5e2489ts2vhll,Funny that the people of our people will have ...,1
2,fav48xfxollnr5ho5pf5e2489ts2vhll,If dummy Bill Kristol has been doing from the ...,1
3,fav48xfxollnr5ho5pf5e2489ts2vhll,Isn't it sad that on a Twitter rant. She is a ...,1
4,fav48xfxollnr5ho5pf5e2489ts2vhll,.#USER# Your story about my management style &...,1


In [7]:
train['label'].value_counts()

0    15000
1    14999
Name: label, dtype: int64

In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19999 entries, 0 to 19998
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   user    19999 non-null  object
 1   tweet   19999 non-null  object
 2   label   19999 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 625.0+ KB


In [9]:
test['label'].value_counts()

1    10000
0     9999
Name: label, dtype: int64

In [5]:
train['tweet'] = train['tweet'].apply(lambda x: clean(x))
test['tweet'] = test['tweet'].apply(lambda x: clean(x))

In [6]:
train = train.groupby(['user', 'label'])['tweet'].agg(list).reset_index()
test = test.groupby(['user', 'label'])['tweet'].agg(list).reset_index()

In [7]:
all_sentences = pd.Series.append(train['tweet'].copy(), test['tweet'].copy())

/tmp/ipykernel_4370/4018911843.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_sentences = pd.Series.append(train['tweet'].copy(), test['tweet'].copy())


In [8]:
sentences = all_sentences.reset_index()['tweet']

In [9]:
sentence_transformer = SentenceTransformer('all-MiniLM-L6-v2')

In [10]:
embeddings = sentence_transformer.encode(sentences)

In [11]:
train['embedding'] = np.copy(embeddings[:300, :]).tolist()
test['embedding'] = np.copy(embeddings[300:, :]).tolist()

In [12]:
x_train = train['embedding'].copy().values
y_train = train['label'].copy().values

x_test = test['embedding'].copy().values
y_test = test['label'].copy().values

In [13]:
x = np.append(x_train, x_test)
y = np.append(y_train, y_test)

In [14]:
x = np.array([np.array(emb) for emb in x])

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.4, shuffle=True, stratify=y, random_state=0)

In [16]:
x_train_tmp = tf.expand_dims(x_train.copy(), -1)

2022-07-03 11:24:25.801330: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-03 11:24:26.640255: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-03 11:24:26.640646: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-03 11:24:26.675855: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

# Models

# Model1

In [17]:
model1 = Sequential()

# model1.add(Input(shape=(x_train.shape[1],1)))

model1.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1), input_shape=[384, 1]))

model1.add(Flatten())

model1.add(Dense(64, activation='relu'))
model1.add(Dense(32, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))


model1.compile(optimizer=Adam(learning_rate=.001), loss='binary_crossentropy', metrics=['accuracy'])

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 384, 128)          4224      
                                                                 
 flatten (Flatten)           (None, 49152)             0         
                                                                 
 dense (Dense)               (None, 64)                3145792   
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3,152,129
Trainable params: 3,152,129
Non-trainable params: 0
_________________________________________________________________


In [18]:
model1.fit(x_train_tmp, y_train.reshape(-1,1), epochs=20, batch_size=16, verbose=2)

Epoch 1/20


2022-07-03 11:25:03.537959: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


19/19 - 29s - loss: 0.6921 - accuracy: 0.5233 - 29s/epoch - 2s/step
Epoch 2/20
19/19 - 0s - loss: 0.6403 - accuracy: 0.6833 - 153ms/epoch - 8ms/step
Epoch 3/20
19/19 - 0s - loss: 0.5254 - accuracy: 0.7500 - 130ms/epoch - 7ms/step
Epoch 4/20
19/19 - 0s - loss: 0.4136 - accuracy: 0.8367 - 114ms/epoch - 6ms/step
Epoch 5/20
19/19 - 0s - loss: 0.3145 - accuracy: 0.9000 - 113ms/epoch - 6ms/step
Epoch 6/20
19/19 - 0s - loss: 0.2481 - accuracy: 0.8967 - 112ms/epoch - 6ms/step
Epoch 7/20
19/19 - 0s - loss: 0.1462 - accuracy: 0.9600 - 114ms/epoch - 6ms/step
Epoch 8/20
19/19 - 0s - loss: 0.1073 - accuracy: 0.9800 - 113ms/epoch - 6ms/step
Epoch 9/20
19/19 - 0s - loss: 0.0628 - accuracy: 0.9900 - 112ms/epoch - 6ms/step
Epoch 10/20
19/19 - 0s - loss: 0.0571 - accuracy: 0.9767 - 110ms/epoch - 6ms/step
Epoch 11/20
19/19 - 0s - loss: 0.0430 - accuracy: 0.9933 - 112ms/epoch - 6ms/step
Epoch 12/20
19/19 - 0s - loss: 0.0257 - accuracy: 1.0000 - 112ms/epoch - 6ms/step
Epoch 13/20
19/19 - 0s - loss: 0.0094 

In [19]:
y_pred = model1.predict(x_test)
ypred = y_pred.reshape(len(y_pred), )
y_pred = np.round(y_pred)
print(classification_report(y_test, y_pred))

7/7 [==============================] - 0s 23ms/step
              precision    recall  f1-score   support

           0       0.62      0.59      0.61       100
           1       0.61      0.64      0.62       100

    accuracy                           0.61       200
   macro avg       0.62      0.61      0.61       200
weighted avg       0.62      0.61      0.61       200



# Model2

In [25]:
model2 = Sequential()

# model1.add(Input(shape=(x_train.shape[1],1)))

model2.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1), input_shape=[384, 1]))
model2.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1)))
model2.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1)))

model2.add(Flatten())

model2.add(Dense(64, activation='relu'))
model2.add(Dense(32, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))


model2.compile(optimizer=Adam(learning_rate=.001), loss='binary_crossentropy', metrics=['accuracy'])

model2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_7 (Conv1D)           (None, 384, 128)          4224      
                                                                 
 conv1d_8 (Conv1D)           (None, 384, 128)          524416    
                                                                 
 conv1d_9 (Conv1D)           (None, 384, 128)          524416    
                                                                 
 flatten_3 (Flatten)         (None, 49152)             0         
                                                                 
 dense_9 (Dense)             (None, 64)                3145792   
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 1)                

In [26]:
model2.fit(x_train_tmp, y_train.reshape(-1,1), epochs=20, batch_size=16, verbose=2)

Epoch 1/20
19/19 - 1s - loss: 0.6986 - accuracy: 0.5233 - 1s/epoch - 53ms/step
Epoch 2/20
19/19 - 1s - loss: 0.6968 - accuracy: 0.5167 - 521ms/epoch - 27ms/step
Epoch 3/20
19/19 - 1s - loss: 0.7046 - accuracy: 0.4667 - 521ms/epoch - 27ms/step
Epoch 4/20
19/19 - 1s - loss: 0.6964 - accuracy: 0.5000 - 517ms/epoch - 27ms/step
Epoch 5/20
19/19 - 1s - loss: 0.6949 - accuracy: 0.4433 - 519ms/epoch - 27ms/step
Epoch 6/20
19/19 - 1s - loss: 0.6941 - accuracy: 0.5133 - 517ms/epoch - 27ms/step
Epoch 7/20
19/19 - 1s - loss: 0.6862 - accuracy: 0.5400 - 518ms/epoch - 27ms/step
Epoch 8/20
19/19 - 1s - loss: 0.6710 - accuracy: 0.6167 - 517ms/epoch - 27ms/step
Epoch 9/20
19/19 - 1s - loss: 0.7159 - accuracy: 0.6033 - 517ms/epoch - 27ms/step
Epoch 10/20
19/19 - 1s - loss: 1.2342 - accuracy: 0.5200 - 517ms/epoch - 27ms/step
Epoch 11/20
19/19 - 1s - loss: 0.7924 - accuracy: 0.5300 - 515ms/epoch - 27ms/step
Epoch 12/20
19/19 - 1s - loss: 0.9245 - accuracy: 0.4867 - 517ms/epoch - 27ms/step
Epoch 13/20
19/1

In [27]:
y_pred = model2.predict(x_test)
ypred = y_pred.reshape(len(y_pred), )
y_pred = np.round(y_pred)
print(classification_report(y_test, y_pred))

7/7 [==============================] - 0s 9ms/step
              precision    recall  f1-score   support

           0       0.58      0.56      0.57       100
           1       0.58      0.60      0.59       100

    accuracy                           0.58       200
   macro avg       0.58      0.58      0.58       200
weighted avg       0.58      0.58      0.58       200



In [135]:
x_test_tmp = tf.expand_dims(x_test.copy(), -1)

In [139]:
y_pred = model2.predict(x_test_tmp)
ypred = y_pred.reshape(len(y_pred), )
y_pred = np.round(y_pred)
print(classification_report(y_test, y_pred))

7/7 [==============================] - 1s 98ms/step
              precision    recall  f1-score   support

           0       0.56      0.62      0.59       100
           1       0.57      0.51      0.54       100

    accuracy                           0.56       200
   macro avg       0.57      0.56      0.56       200
weighted avg       0.57      0.56      0.56       200



# Model3

# More Conv1D layers. No Dense layers.

In [28]:
model3 = Sequential()

# model1.add(Input(shape=(x_train.shape[1],1)))

model3.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1), input_shape=[384, 1]))
model3.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1)))
model3.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1)))
model3.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1)))
model3.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1)))

model3.add(Flatten())

# model3.add(Dense(64, activation='relu'))
# model3.add(Dense(32, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))


model3.compile(optimizer=Adam(learning_rate=.001), loss='binary_crossentropy', metrics=['accuracy'])

model3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_10 (Conv1D)          (None, 384, 128)          4224      
                                                                 
 conv1d_11 (Conv1D)          (None, 384, 128)          524416    
                                                                 
 conv1d_12 (Conv1D)          (None, 384, 128)          524416    
                                                                 
 conv1d_13 (Conv1D)          (None, 384, 128)          524416    
                                                                 
 conv1d_14 (Conv1D)          (None, 384, 128)          524416    
                                                                 
 flatten_4 (Flatten)         (None, 49152)             0         
                                                                 
 dense_12 (Dense)            (None, 1)                

In [29]:
model3.fit(x_train_tmp, y_train.reshape(-1,1), epochs=20, batch_size=16, verbose=2)

Epoch 1/20
19/19 - 1s - loss: 0.7168 - accuracy: 0.4867 - 1s/epoch - 75ms/step
Epoch 2/20
19/19 - 1s - loss: 0.6936 - accuracy: 0.4733 - 932ms/epoch - 49ms/step
Epoch 3/20
19/19 - 1s - loss: 0.7006 - accuracy: 0.5367 - 918ms/epoch - 48ms/step
Epoch 4/20
19/19 - 1s - loss: 0.7949 - accuracy: 0.5233 - 926ms/epoch - 49ms/step
Epoch 5/20
19/19 - 1s - loss: 0.6963 - accuracy: 0.5333 - 898ms/epoch - 47ms/step
Epoch 6/20
19/19 - 1s - loss: 0.7005 - accuracy: 0.4933 - 937ms/epoch - 49ms/step
Epoch 7/20
19/19 - 1s - loss: 0.7260 - accuracy: 0.5133 - 935ms/epoch - 49ms/step
Epoch 8/20
19/19 - 1s - loss: 0.7085 - accuracy: 0.5533 - 929ms/epoch - 49ms/step
Epoch 9/20
19/19 - 1s - loss: 0.7218 - accuracy: 0.4400 - 891ms/epoch - 47ms/step
Epoch 10/20
19/19 - 1s - loss: 0.6938 - accuracy: 0.5333 - 886ms/epoch - 47ms/step
Epoch 11/20
19/19 - 1s - loss: 0.6958 - accuracy: 0.5067 - 889ms/epoch - 47ms/step
Epoch 12/20
19/19 - 1s - loss: 0.6754 - accuracy: 0.5600 - 889ms/epoch - 47ms/step
Epoch 13/20
19/1

In [31]:
y_pred = model3.predict(x_test)
ypred = y_pred.reshape(len(y_pred), )
y_pred = np.round(y_pred)
print(classification_report(y_test, y_pred))

7/7 [==============================] - 0s 15ms/step
              precision    recall  f1-score   support

           0       0.53      0.49      0.51       100
           1       0.53      0.57      0.55       100

    accuracy                           0.53       200
   macro avg       0.53      0.53      0.53       200
weighted avg       0.53      0.53      0.53       200



# Removing the Dense layers, makes the netowrk unable to learn.

# Model4

# One Conv1D layer, more dense layers. Epochs=50.

In [37]:
model4 = Sequential()

# model1.add(Input(shape=(x_train.shape[1],1)))

model4.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1), input_shape=[384, 1]))

model4.add(Flatten())

model4.add(Dense(128, activation='relu'))
model4.add(Dense(64, activation='relu'))
model4.add(Dense(32, activation='relu'))
model4.add(Dense(1, activation='sigmoid'))


model4.compile(optimizer=Adam(learning_rate=.001), loss='binary_crossentropy', metrics=['accuracy'])

model4.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_17 (Conv1D)          (None, 384, 128)          4224      
                                                                 
 flatten_7 (Flatten)         (None, 49152)             0         
                                                                 
 dense_21 (Dense)            (None, 128)               6291584   
                                                                 
 dense_22 (Dense)            (None, 64)                8256      
                                                                 
 dense_23 (Dense)            (None, 32)                2080      
                                                                 
 dense_24 (Dense)            (None, 1)                 33        
                                                                 
Total params: 6,306,177
Trainable params: 6,306,177
No

In [38]:
model4.fit(x_train_tmp, y_train.reshape(-1,1), epochs=50, batch_size=16, verbose=2)

Epoch 1/50
19/19 - 1s - loss: 0.6920 - accuracy: 0.5300 - 533ms/epoch - 28ms/step
Epoch 2/50
19/19 - 0s - loss: 0.6242 - accuracy: 0.7067 - 146ms/epoch - 8ms/step
Epoch 3/50
19/19 - 0s - loss: 0.5655 - accuracy: 0.7233 - 142ms/epoch - 7ms/step
Epoch 4/50
19/19 - 0s - loss: 0.4923 - accuracy: 0.7933 - 141ms/epoch - 7ms/step
Epoch 5/50
19/19 - 0s - loss: 0.3221 - accuracy: 0.8800 - 141ms/epoch - 7ms/step
Epoch 6/50
19/19 - 0s - loss: 0.2216 - accuracy: 0.9200 - 139ms/epoch - 7ms/step
Epoch 7/50
19/19 - 0s - loss: 0.1501 - accuracy: 0.9433 - 147ms/epoch - 8ms/step
Epoch 8/50
19/19 - 0s - loss: 0.0854 - accuracy: 0.9700 - 140ms/epoch - 7ms/step
Epoch 9/50
19/19 - 0s - loss: 0.0350 - accuracy: 0.9900 - 134ms/epoch - 7ms/step
Epoch 10/50
19/19 - 0s - loss: 0.0137 - accuracy: 1.0000 - 136ms/epoch - 7ms/step
Epoch 11/50
19/19 - 0s - loss: 0.0040 - accuracy: 1.0000 - 139ms/epoch - 7ms/step
Epoch 12/50
19/19 - 0s - loss: 0.0015 - accuracy: 1.0000 - 134ms/epoch - 7ms/step
Epoch 13/50
19/19 - 0s -

In [39]:
y_pred = model4.predict(x_test)
ypred = y_pred.reshape(len(y_pred), )
y_pred = np.round(y_pred)
print(classification_report(y_test, y_pred))

7/7 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

           0       0.61      0.58      0.59       100
           1       0.60      0.63      0.61       100

    accuracy                           0.60       200
   macro avg       0.61      0.60      0.60       200
weighted avg       0.61      0.60      0.60       200



# Increasing the number of epochs does not improve performance.

# Model5

# Adding LSTM layer to model1

In [2]:
model5 = Sequential()

# model1.add(Input(shape=(x_train.shape[1],1)))

model5.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1), input_shape=[384, 1]))

model5.add(LSTM(128))

model5.add(Flatten())



model5.add(Dense(64, activation='relu'))
model5.add(Dense(32, activation='relu'))
model5.add(Dense(1, activation='sigmoid'))


model5.compile(optimizer=Adam(learning_rate=.001), loss='binary_crossentropy', metrics=['accuracy'])

model5.summary()

2022-08-13 11:05:37.000017: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-13 11:05:37.000073: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (petar-X580VD): /proc/driver/nvidia/version does not exist
2022-08-13 11:05:37.072842: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 384, 128)          4224      
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 146,177
Trainable params: 146,177
Non-trai

In [55]:
model5.fit(x_train_tmp, y_train.reshape(-1,1), epochs=20, batch_size=16, verbose=2)

Epoch 1/20
19/19 - 2s - loss: 0.6940 - accuracy: 0.4933 - 2s/epoch - 96ms/step
Epoch 2/20
19/19 - 0s - loss: 0.6934 - accuracy: 0.5000 - 387ms/epoch - 20ms/step
Epoch 3/20
19/19 - 0s - loss: 0.6933 - accuracy: 0.5000 - 387ms/epoch - 20ms/step
Epoch 4/20
19/19 - 0s - loss: 0.6931 - accuracy: 0.5000 - 378ms/epoch - 20ms/step
Epoch 5/20
19/19 - 0s - loss: 0.6932 - accuracy: 0.5000 - 379ms/epoch - 20ms/step
Epoch 6/20
19/19 - 0s - loss: 0.6932 - accuracy: 0.5000 - 406ms/epoch - 21ms/step
Epoch 7/20
19/19 - 0s - loss: 0.6927 - accuracy: 0.5100 - 399ms/epoch - 21ms/step
Epoch 8/20
19/19 - 0s - loss: 0.6925 - accuracy: 0.5000 - 375ms/epoch - 20ms/step
Epoch 9/20
19/19 - 0s - loss: 0.6897 - accuracy: 0.5467 - 392ms/epoch - 21ms/step
Epoch 10/20
19/19 - 0s - loss: 0.6870 - accuracy: 0.5533 - 377ms/epoch - 20ms/step
Epoch 11/20
19/19 - 0s - loss: 0.6828 - accuracy: 0.5800 - 384ms/epoch - 20ms/step
Epoch 12/20
19/19 - 0s - loss: 0.6778 - accuracy: 0.5833 - 392ms/epoch - 21ms/step
Epoch 13/20
19/1

In [56]:
y_pred = model5.predict(x_test)
ypred = y_pred.reshape(len(y_pred), )
y_pred = np.round(y_pred)
print(classification_report(y_test, y_pred))

7/7 [==============================] - 0s 12ms/step
              precision    recall  f1-score   support

           0       0.62      0.71      0.66       100
           1       0.66      0.56      0.61       100

    accuracy                           0.64       200
   macro avg       0.64      0.64      0.63       200
weighted avg       0.64      0.64      0.63       200



# Adding one LSTM layer does not improve performance.

# Model6

# Adding multiple LSTM layers

In [63]:
model6 = Sequential()

# model1.add(Input(shape=(x_train.shape[1],1)))

model6.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1), input_shape=[384, 1]))

model6.add(LSTM(128, return_sequences=True))
model6.add(LSTM(64, return_sequences=True))
model6.add(LSTM(32))

# model6.add(Flatten())

model6.add(Dense(128, activation='relu'))
model6.add(Dense(64, activation='relu'))
model6.add(Dense(32, activation='relu'))
model6.add(Dense(1, activation='sigmoid'))


model6.compile(optimizer=Adam(learning_rate=.01), loss='binary_crossentropy', metrics=['accuracy'])

model6.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_28 (Conv1D)          (None, 384, 128)          4224      
                                                                 
 lstm_14 (LSTM)              (None, 384, 128)          131584    
                                                                 
 lstm_15 (LSTM)              (None, 384, 64)           49408     
                                                                 
 lstm_16 (LSTM)              (None, 32)                12416     
                                                                 
 dense_51 (Dense)            (None, 128)               4224      
                                                                 
 dense_52 (Dense)            (None, 64)                8256      
                                                                 
 dense_53 (Dense)            (None, 32)              

In [64]:
model6.fit(x_train_tmp, y_train.reshape(-1,1), epochs=20, batch_size=16, verbose=2)

Epoch 1/20
19/19 - 4s - loss: 0.7008 - accuracy: 0.5200 - 4s/epoch - 224ms/step
Epoch 2/20
19/19 - 1s - loss: 0.6945 - accuracy: 0.5000 - 816ms/epoch - 43ms/step
Epoch 3/20
19/19 - 1s - loss: 0.6948 - accuracy: 0.4533 - 787ms/epoch - 41ms/step
Epoch 4/20
19/19 - 1s - loss: 0.6944 - accuracy: 0.5000 - 817ms/epoch - 43ms/step
Epoch 5/20
19/19 - 1s - loss: 0.6939 - accuracy: 0.4533 - 791ms/epoch - 42ms/step
Epoch 6/20
19/19 - 1s - loss: 0.6959 - accuracy: 0.5000 - 792ms/epoch - 42ms/step
Epoch 7/20
19/19 - 1s - loss: 0.6938 - accuracy: 0.5000 - 851ms/epoch - 45ms/step
Epoch 8/20
19/19 - 1s - loss: 0.6936 - accuracy: 0.4933 - 798ms/epoch - 42ms/step
Epoch 9/20
19/19 - 1s - loss: 0.6934 - accuracy: 0.4933 - 823ms/epoch - 43ms/step
Epoch 10/20
19/19 - 1s - loss: 0.6937 - accuracy: 0.4533 - 778ms/epoch - 41ms/step
Epoch 11/20
19/19 - 1s - loss: 0.6938 - accuracy: 0.4667 - 817ms/epoch - 43ms/step
Epoch 12/20
19/19 - 1s - loss: 0.6936 - accuracy: 0.4800 - 850ms/epoch - 45ms/step
Epoch 13/20
19/

In [65]:
y_pred = model6.predict(x_test)
ypred = y_pred.reshape(len(y_pred), )
y_pred = np.round(y_pred)
print(classification_report(y_test, y_pred))

7/7 [==============================] - 1s 26ms/step
              precision    recall  f1-score   support

           0       0.50      1.00      0.67       100
           1       0.00      0.00      0.00       100

    accuracy                           0.50       200
   macro avg       0.25      0.50      0.33       200
weighted avg       0.25      0.50      0.33       200



/home/petar/jupyter/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/petar/jupyter/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/petar/jupyter/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# With more LSTM layers the network does not even learn.